<a href="https://colab.research.google.com/github/kumar2612/ai_ml/blob/ai_ml_basics_nov2024/langgraph_tutorials/1_Build_a_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain-core langgraph>0.2.27

In [4]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 3.1 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
import os

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]=userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_PROJECT"]="langchain_tutorials"

In [6]:
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.3-70b-versatile")

In [7]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content = 'My Name is Ashish')])

AIMessage(content="Hello Ashish, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 40, 'total_tokens': 67, 'completion_time': 0.098181818, 'prompt_time': 0.007728087, 'queue_time': 0.0076722119999999994, 'total_time': 0.105909905}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_fcc3b74982', 'finish_reason': 'stop', 'logprobs': None}, id='run-f08c587b-cf1e-40ad-a493-449d78ea31c3-0', usage_metadata={'input_tokens': 40, 'output_tokens': 27, 'total_tokens': 67})

In [8]:
model.invoke([HumanMessage(content = 'What is my name')])

AIMessage(content="I don't know your name. I'm a large language model, I don't have the ability to access personal information about you, including your name. Our conversation just started, and I don't have any prior knowledge about you. If you'd like to share your name, I'd be happy to chat with you and use it in our conversation!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 39, 'total_tokens': 111, 'completion_time': 0.261818182, 'prompt_time': 0.007763348, 'queue_time': 0.006602210000000001, 'total_time': 0.26958153}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_4e32347616', 'finish_reason': 'stop', 'logprobs': None}, id='run-186ad972-bb6b-441d-b20d-e59133e21834-0', usage_metadata={'input_tokens': 39, 'output_tokens': 72, 'total_tokens': 111})

In [12]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Ashish"),
        AIMessage(content="Hello Ashish! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Ashish.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 67, 'total_tokens': 74, 'completion_time': 0.025454545, 'prompt_time': 0.011417101, 'queue_time': 0.085393705, 'total_time': 0.036871646}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_4e32347616', 'finish_reason': 'stop', 'logprobs': None}, id='run-2748fd18-bebb-4607-b064-d3b797934d65-0', usage_metadata={'input_tokens': 67, 'output_tokens': 7, 'total_tokens': 74})

In [26]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema = MessagesState)

def call_model(state: MessagesState):
  response = model.invoke(state["messages"])
  return {"messages" : response}

workflow.add_edge(START,"model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer = memory)

In [17]:
config = {"configurable": {"thread_id": "abc123"}}

In [27]:
query = "Hello! I am Ashish!"

input_messages = [HumanMessage(content = query)]

output = app.invoke({"messages":input_messages}, config = config)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hello Ashish! It's nice to meet you. Is there something I can help you with or would you like to chat?


Exception ignored in: <coroutine object Pregel.ainvoke at 0x7ef0fccaf6f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py", line 27, in run
KeyError: '__builtins__'
Exception ignored in: <coroutine object Pregel.ainvoke at 0x7ef0fccaf6f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py", line 27, in run
KeyError: '__builtins__'


In [28]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Ashish. You told me that when we started chatting.


In [29]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't know your name. I'm a large language model, I don't have the ability to access personal information about you, including your name. I'm here to help answer your questions and provide information, but I don't have any prior knowledge about you. If you'd like to share your name with me, I'd be happy to chat with you!


In [30]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Ashish. You mentioned it at the beginning of our conversation.


In [33]:
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

config = {"configurable": {"thread_id": "abc123"}}
query = ' Hi I am Ashish'
input_messages = [HumanMessage(query)]
# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hello Ashish, it's nice to meet you. Is there something I can help you with or would you like to chat?


In [34]:
config = {"configurable": {"thread_id": "abc123"}}
query = ' Who am I'
input_messages = [HumanMessage(query)]
# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You are Ashish, the person I'm currently chatting with. If you'd like to share more about yourself, I'm all ears! What do you do, or what are your interests?


In [35]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [37]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [42]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arrr, yer name be Jim, matey! I remember ye tellin' me that before, and I be recallin' it now. No need to introduce yerself again, Jim, I be knowin' ye already!


In [43]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ye be askin' again, eh? Alright then, matey! Yer name be... JIM! Yer a stubborn one, ain't ye, Jim?


In [44]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [45]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [47]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Hindi"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

नमस्ते बॉब! मैं आपकी मदद करने के लिए तैयार हूँ। आज मैं आपके लिए क्या कर सकता हूँ?


In [48]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

आपका नाम बॉब है।


In [66]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=200,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [67]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [55]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't know your name. You haven't told me what it is. Would you like to share it with me?


In [56]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked me to solve the math problem "2 + 2".


In [68]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Hi| Todd|,| here|'s| one|:| What| do| you| call| a| fake| nood|le|?| An| imp|asta|.| I| hope| that| made| you| laugh|.| Do| you| want| to| hear| another| one|?||

In [70]:
query = "What did you tell me before"
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Before|,| I| told| you| a| joke| about| a| fake| nood|le| being| called| an| "|imp|asta|".| I| also| offered| to| tell| you| another| joke| if| you| wanted| to| hear| one|.||